# Deep Learning Project: Pet Classifier using CNN

Prepration
- Extract the ipynb file and the data in the same folder

Data Set
- A production grade program as 10,000 training images
- This is a small program with 20 images of cats and 20 images of dogs. 
- The evaluation set has 10 images of cats and 10 images of dogs

Runs
- The student is expected to run the 100-300 training step
- A production grade code would have about 20k-50k training steps

## Name : Anant Kumar Srirangam
## Date   : 24 Mar 2019

### Import modules

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
import sys


# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

### Set hyper parameters
- Run the program with three num_steps : 100,200,300

In [ ]:
reset_graph()

img_size = 32
num_channels = 3
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)
trainpath='./data/train'
testpath='./data/test'
labels = {'cats': 0, 'dogs': 1}
fc_size=32 #size of the output of final FC layer
num_steps=300 #Try 100, 200, 300. number of steps that training data should be looped. Usually 20K
tf.logging.set_verbosity(tf.logging.INFO)

### Read the image dataset

In [ ]:
def read_images_classes(basepath,imgSize=img_size):
    image_stack = []
    label_stack = []

    for counter, l in enumerate(labels):
        path = os.path.join(basepath, l,'*g')
        for img in glob.glob(path):
            one_hot_vector =np.zeros(len(labels),dtype=np.int16)
            one_hot_vector[counter]=1
            image = cv2.imread(img)
            im_resize = cv2.resize(image,img_shape, interpolation=cv2.INTER_CUBIC)
            image_stack.append(im_resize)
            label_stack.append(labels[l])            
    return np.array(image_stack), np.array(label_stack)

X_train, y_train=read_images_classes(trainpath)
X_test, y_test=read_images_classes(testpath)

#test a sample image
print('length of train image set',len(X_train))
print('X_data shape:', X_train.shape)
print('y_data shape:', y_train.shape)

fig1 = plt.figure() 
ax1 = fig1.add_subplot(2,2,1) 
img = cv2.resize(X_train[0],(64,64), interpolation=cv2.INTER_CUBIC)
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title(y_train[0])
plt.show()

In [ ]:
def cnn_model_fn(features, labels, mode):
    print (labels.shape)
    ## No information about padding was given hence using 'same'
    
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    
    # First Convolution Layer --- 32 Kernels
    c1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding="same",activation=tf.nn.relu)
    
    # First Pooling Layer
    p1 = tf.layers.max_pooling2d(inputs=c1,pool_size=[2,2],strides=2)
    
    # Second Convolution Layer --- 64 Kernels
    c2 = tf.layers.conv2d(inputs=p1,filters=64,kernel_size=[5,5],padding="same",activation=tf.nn.relu)
    
    # Second Pooling Layer
    p2 = tf.layers.max_pooling2d(inputs=c2,pool_size=[2,2],strides=2)
    
    # Flattening the last pooling layer
    pool_flat = tf.contrib.layers.flatten(p2)
    
    # Dense Layer --- fc_size = 32 (units)
    dense = tf.layers.dense(inputs=pool_flat,units=32,activation=tf.nn.relu)
    
    # Adding Dropout --- rate = 0.4
    dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits layer // Output Layer --- classes = 2 (units)
    logits = tf.layers.dense(inputs=dropout,units=2)
    
    predictions ={
        "classes": tf.argmax(logits, axis=1),
        "probabilities": tf.nn.softmax(logits,name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode ,predictions=predictions)
    
    # Loss Function
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    # Training Mode Optimizer For Estimator --- Learning Rate = 10^-3
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    # Evaluation Metrics for EVAL mode
    eval_metric_ops = {
        "accuracy":tf.metrics.accuracy(labels=labels,predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)
    

### Run the tensorflow model

This section will use the model defined by the student and run the training and evaluation step

In [ ]:
#X_train = np.array((X_train/255.0),dtype=np.float16)
#X_test = np.array((X_test/255.0), dtype=np.float16)
X_train = np.array((X_train/255.0),dtype=np.float32)
X_test = np.array((X_test/255.0), dtype=np.float32)

pets_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/pets_convnet_model")
#pets_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn)
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_train}, y=y_train, batch_size=10,
                                                      num_epochs=None, shuffle=True)
pets_classifier.train(input_fn=train_input_fn, steps=num_steps, hooks=[logging_hook])
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X_test}, y=y_test, num_epochs=1,shuffle=False)
eval_results = pets_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

### Accuracy = 0.5 
### Loss = 0.69

Loss was minimized using gradient descent optimizer